In [21]:
#Initial imports
import numpy as np
import pandas as pd
import hvplot.pandas
import alpaca_trade_api as tradeapi
import random as random
#Ignore verbose warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [22]:
#Function to simulate weights for the stocks to use in portfolio construction. It takes the list of tickers as input and the # of combinations the user would like to use
def simulate_weights(tickers, no_of_trials):
    numOfAssets=len(tickers)
    trials = no_of_trials
    overall_weights=[]
    total = 0
    for i in range(trials):
        weights=[]
        for i in range(numOfAssets):
            weights.append(random.random())
        total = sum(weights)
        for i in range(len(weights)):
            weights[i]=round(weights[i]/total,2)
        overall_weights.append(weights)
    #return weights list
    return(overall_weights)

In [23]:
#Function to calculate the average return and the std_deviation of the stock prices for use in portfolio contruction.
def calculate_stock_metrics(tickers):
    start_date = '2020-01-01'
    end_date = '2020-12-31'
    timeframe = '1D'
    alpaca_api_key = 'PKURMS5ELMEHV4ZGBXTQ'
    alpaca_secret_key = 'NhYNWSU4szskB7qUMl2FXPqYoQZXwL50ahf54jds'
    # Create the Alpaca API object
    alpaca = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version="v2")
    # Get current price data for tickers
    alpaca_df = alpaca.get_bars(
        tickers,
        timeframe,
        start = start_date,
        end = end_date
    ).df
    #Clean the stock data and store in a separate dataframe for analysis
    alpaca_df = alpaca_df.drop(columns=['vwap','trade_count','open', 'high', 'low', 'volume'])
    stock_df = pd.DataFrame(columns=['ticker','mean_close','mean_return'])#,'std_dev'])
    counter = 0
    for ticker in tickers:
        #record the stock in stock_df
        stock_df.at[counter, 'ticker'] = ticker
        #calculate the mean price for the ticker
        stock_df.at[counter,'mean_return'] = alpaca_df.groupby('symbol').get_group(ticker)['close'].pct_change().mean()
        #calculate the mean daily return for the ticker
        stock_df.at[counter,'mean_close'] = alpaca_df.groupby('symbol').get_group(ticker)['close'].mean()
        #calculate the standard deviation of the daily return for the ticker
        #stock_df.at[counter, 'std_dev'] = alpaca_df.groupby('symbol').get_group(ticker)['close'].pct_change().std()
        counter += 1
    return(stock_df)

In [24]:
#hardcoded variables to be replaced by user input and environment variable calls in the future
portfolio_stocks = ['MSFT','GOOG', 'AAPL', 'AMZN', 'TSLA']
time_horizon = 365
investment_amount = 100000
no_of_trials = 1
portfolio_weights = simulate_weights(portfolio_stocks,no_of_trials)
portfolio_investment = np.array(portfolio_weights) * investment_amount
stock_df = calculate_stock_metrics(portfolio_stocks)

In [25]:
#loop through each row in the stock_df and calculate value of portfolio by multiplying the equivalent weight and the mean_close
stock_df['investment_amount'] = 0
stock_df['weight'] = 0
for i in range(len(stock_df)):
    stock_df.at[i,'weight'] = portfolio_weights[0][i]
    stock_df.at[i,'investment_amount'] = portfolio_investment[0][i]
    #stock_df.at[i,'mean_close']*portfolio_weights[0][i]
#Convert the investment_amount and weight columns to float
stock_df['investment_amount'] = stock_df['investment_amount'].astype(float)
stock_df['weight'] = stock_df['weight'].astype(float)
#Display the stock_df
stock_df.head()

,ticker,mean_close,mean_return,investment_amount,weight
0,MSFT,193.038722,0.001672,38000.0,0.38
1,GOOG,1481.406933,0.001275,12000.0,0.12
2,AAPL,259.905933,-0.000384,9000.0,0.09
3,AMZN,2680.86336,0.002437,26000.0,0.26
4,TSLA,786.649684,0.006385,14000.0,0.14
